<a href="https://colab.research.google.com/github/katamaiah/shopassist-assignment/blob/main/ShopAssist_2_0_Assignment_Katamaiah_ACPGENC8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ShopAssist AI 2.0

Combined function calling API with the ShopAssist AI 2.0's system design to make it more seamless,  less lines of code and cost efficient

This solution  redefines the overall conversation flow. Also, the output of the layers are now more consistent and therefore, require less manipulation

This solution assumes the input as updated_laptop.csv which has the python dictionalry filled in the last column ,hence no seperate product_map_layer is implemented in this solution as the focus is on function calling API prototype only.

This solution uses openai model gpt4o and tools functionality to enable function calling

To test this have the updated_laptop.csv and OpenAI_API_Key.txt files in your google drive folder SHOP_ASSIST_ASSIGNMENT under MyDrive

Conversations history is written to a text file for further references

In [ ]:
#Install openai
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/SHOP_ASSIST_ASSIGNMENT')
!ls
## Import the necessary libraries for building the project
import os,json,ast
import openai
from openai import OpenAI
import pandas as pd
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored



## Add your OPENAI API KEY
openai.api_key = open("OpenAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

Mounted at /content/drive
conversation.txt  laptop_data.csv  OpenAI_API_Key.txt  updated_laptop.csv


Mounted at /content/drive


IMPROVEMENT#1 - Introducing openAI Function calling API to minimize the manipulations and api calls

In [ ]:
## Function Description for the Function Calling API
tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_laptop_recommendations",
                    "description": "Get the top 3 laptops from the catalogue, that best matches what the user is asking based on 'GPU intensity','Display quality','Portability','Multitasking','Processing speed' & 'Budget",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "gpu intensity": {
                                "type": "string",
                                "description": "The requirement of the user in GPU capacity classfied as low, medium or high" ,
                            },
                            "display quality": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's Display Quality & capacity classfied as low, medium or high" ,
                            },
                            "portability": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's portability classfied as low, medium or high" ,
                            },
                            "multitasking": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's Multitasking classfied as low, medium or high" ,
                            },
                            "processing speed": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's Processing speed classfied as low, medium or high" ,
                            },
                            "budget": {
                                "type": "integer",
                                "description": "The maximum budget of the user" ,
                            },

                        },
                        "required": ["GPU intensity","Display quality","Portability","Multitasking","Processing speed","Budget"],
                    },
                }
            }
        ]

IMPROVEMENT#2 - This initializes the variable conversation with the system message. Uses an improved prompt with 50% reduced tokens than the original prompt with simplified ,streamlined instructions and explicit chain of thougts

In [ ]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'GPU intensity': "high",
                        'Display quality':"high",
                        'Portability': "low",
                        'Multitasking': "high",
                        'Processing speed': "high",
                        'Budget': "150000"}


    system_message = f"""
        You are an intelligent laptop gadget expert. Your goal is to find the best laptop for a user by asking relevant questions and understanding their profile. You need to confidently fill the values for the keys ('GPU intensity', 'Display quality', 'Portability', 'Multitasking', 'Processing speed', 'Budget') in the Python dictionary based on the user's responses.

        The Python dictionary should look like this:
        {{'GPU intensity': 'values', 'Display quality': 'values', 'Portability': 'values', 'Multitasking': 'values', 'Processing speed': 'values', 'Budget': 'values'}}

        Instructions for filling the dictionary:
        - The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance stated by the user.
        - The value for 'Budget' should be a numerical value extracted from the user's response and must be greater than or equal to 25000 INR. If the budget is below this, inform the user there are no laptops in that range.
        - Do not randomly assign values. Ensure all values are inferred from the user's responses.

        Chain of Thoughts:
        1. Ask questions to understand the user's profile and requirements. Focus on determining the values for the keys ('GPU intensity', 'Display quality', 'Portability', 'Multitasking', 'Processing speed', 'Budget'). If unclear, ask follow-up questions.
        2. For keys not filled in the previous step, ask specific questions to gather the necessary information. Use logical questions rather than directly citing the key.
        3. Verify if you have correctly filled the values for all keys. If uncertain, ask clarifying questions.

        Example User Interaction:
        User: "Hi, I am an editor."
        Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. You would need high multitasking capability and a high-end display for better visuals. Are you more involved in video editing, photo editing, or both?"
        User: "I primarily work with After Effects."
        Assistant: "Working with After Effects involves graphics, animations, and rendering, requiring a high GPU. Do you work with high-resolution media files, such as 4K videos?"
        User: "Yes, sometimes I work with 4K videos."
        Assistant: "Processing 4K videos requires a good processor and high GPU. Do you need a lightweight laptop for frequent travel or work primarily from a stationary location?"
        User: "I travel but do not carry my laptop."
        Assistant: "Please let me know your budget for the laptop."
        User: "My max budget is 1.5 lakh INR."
        Assistant: "{example_user_dict}"

        Start with a short welcome message and encourage the user to share their requirements.
        """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

IMPROVEMENT#3 - Chat completions api modified to use function calling using tools . this was referred as functions in gpt3.5

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))

def get_chat_model_completions(messages,tools=None, tool_choice=None, model="gpt-4o"):
    client = OpenAI()
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0, # this is the degree of randomness of the model's output
            max_tokens = 500,
            tools=tools
        )
        response = json.loads(response.to_json())
        # print(response)
        return response["choices"][0]["message"]
    except Exception as e:
        print(f"Exception: {e}")
        return e

IMPROVEMENT#4 - Provide a color coding to each role so that you can differentiate the queries and responses easily in the chatbot

Extract dictionary from string

In [ ]:
import pandas as pd
import ast
import re

def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

Moderation_check to validate the input for flags . Reduced lines of code to use if else as part of return

In [ ]:
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(user_input):
    client = OpenAI()
    # Call the OpenAI API to perform moderation on the user's input.
    response = client.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    flagged = response.results[0].flagged
    return "Flagged" if flagged else "Not Flagged"

Extract the laptop specs from the descriptions

Added few shot learnings

Compare laptops with user requests

In [ ]:
def compare_laptops_with_user(user_req_string):
  budget = user_req_string.get('budget', '0')
  laptop_df= pd.read_csv('updated_laptop.csv')
  filtered_laptops = laptop_df.copy()
  filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
  filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

  mappings = {
          'low': 0,
          'medium': 1,
          'high': 2
      }
  # Create 'Score' column in the DataFrame and initialize to 0
  filtered_laptops['Score'] = 0
  for index, row in filtered_laptops.iterrows():
      user_product_match_str = row['laptop_feature']
      laptop_values = extract_dictionary_from_string(user_product_match_str)
      score = 0

      for key, user_value in user_req_string.items():
        if key.lower() == 'budget':
            continue  # Skip budget comparison
        laptop_value = laptop_values.get(key, None)
        laptop_mapping = mappings.get(laptop_value.lower(), -1)
        user_mapping = mappings.get(user_value.lower(), -1)
        if laptop_mapping >= user_mapping:
          ### If the laptop value is greater than or equal to the user value the score is incremented by 1
          score += 1

      filtered_laptops.loc[index, 'Score'] = score

  # Sort the laptops by score in descending order and return the top 5 products
  top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
  top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

  return top_laptops.to_json(orient='records')



Validate the recommendations with score

In [ ]:
def recommendation_validation(laptop_recommendation):

    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
      if data[i]['Score'] > 2:
        data1.append(data[i])

    return json.dumps(data1)

IMPROVEMENT #5 - Dialogue Management system with less manipulation and reduced api calls

1.intent_confirmation_layer and dictionary_present replaced by a simple function call to generate the response always as a python dictionary as mentioned in the function call

In [ ]:
def dialogue_mgmt_system():
    # Initialize the conversation and create the assistant object instance
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)["content"]

    print(introduction + '\n')

    user_input = ""
    # Step 1: check moderation
    while(user_input != "exit"):
      user_input = input("")
      moderation = moderation_check(user_input)
      if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
      conversation.append({"role": "user", "content": user_input})

      # Step 2: Pass the user input to GPT
      response_assistant = get_chat_model_completions(conversation,tools=tools)

      if "tool_calls" in list(response_assistant.keys()) and response_assistant["tool_calls"]:
        function_called_message = "\nThank you for your inputs. Please wait, while I fetch relevant products\n"
        conversation.append({"role": "assistant", "content": function_called_message})
        print(function_called_message)

        # Step 3: Extract top3 laptops by calling the external function
        tool  = response_assistant["tool_calls"][0]
        tool_call_id = tool["id"]
        function_name = tool["function"]["name"]
        function_args = json.loads(tool["function"]["arguments"])

        top_3_laptops = compare_laptops_with_user(function_args)
        function_response = recommendation_validation(top_3_laptops)
        if len(function_response) == 0:
          print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
          break
        # Step 4: send the info on the function call and function response to GPT
        conversation.append(response_assistant)
        conversation.append(
            {
                "tool_call_id": tool_call_id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
        recommendation = get_chat_model_completions(conversation)

        conversation.append({"role": "assistant", "content": recommendation["content"]})
        print("\n" +  recommendation["content"] + "\n")
        print("Enter exit to end the conversation or press enter to continue\n")
      else:
        conversation.append({"role": "assistant", "content": response_assistant["content"]})
        print("\n" +  response_assistant["content"] + "\n")



    # Save conversation in a text file
    with open("conversation.txt", "w") as file:
        for message in conversation:
            file.write(f"{message['role']}: {message['content']}\n")

In [ ]:
dialogue_mgmt_system()

Hello! I'm here to help you find the perfect laptop. Could you please share a bit about your primary use for the laptop? For example, are you a gamer, a professional editor, a student, or do you have other specific needs? Additionally, let me know your budget range.

i am a professional editor

Great! As a professional editor, you likely require a laptop that can handle demanding tasks. You would need high multitasking capability and a high-end display for better visuals. Are you more involved in video editing, photo editing, or both?

video editing

Working with video editing involves graphics, animations, and rendering, which require a high GPU. Do you work with high-resolution media files, such as 4K videos?

4k

Processing 4K videos requires a good processor and high GPU. Do you need a lightweight laptop for frequent travel, or do you work primarily from a stationary location?

stationary

Got it. Since you work primarily from a stationary location, portability is less of a concern